In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense

In [ ]:
# 開啟萃取圖檔稍微看一下樣子
img = Image.open("test.bmp")
test_data = np.asarray(img)
plt.imshow(test_data,cmap='gray')
img.close()

hack_img = test_data

In [ ]:
# 載入原本的模組
load_model = tf.keras.models.load_model('.\\mnist_basic_model.h5')
load_model.summary()

In [ ]:
# 想辦法透過 FGSM 演算法針對載入的圖檔做出生成式對抗樣本

# 定義圖片標籤
hack_label = 3
num_classes=10
hack_label = tf.keras.utils.to_categorical(hack_label, num_classes)
hack_label = tf.reshape(hack_label,[1,10])

#轉換輸入圖片
hack_data = hack_img.reshape(1,784)
hack_data = hack_data.astype('float32')
hack_data  = hack_data / 255.0

inputs_tf = tf.convert_to_tensor(hack_data)
labels_tf = tf.convert_to_tensor(hack_label)

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(inputs_tf)
    logits = load_model(inputs_tf, training=False)
    loss = tf.keras.losses.categorical_crossentropy(labels_tf, logits)    

# 實作這種梯度下降的都先稍微看一下有沒有數值, 如果都是 0 有很高的概率做錯了
gradient = tape.gradient(loss, inputs_tf)    
print("gradient:",gradient)

In [ ]:
gradient_signs = tf.sign(gradient)
print("gradient:",gradient_signs)

In [ ]:
# 這個是暴力測試出來的
epsilon = 0.06
perturbed_inputs = tf.add(inputs_tf, epsilon * gradient_signs)
print(perturbed_inputs)

In [ ]:
# Clip the perturbed inputs to ensure they stay within the valid range (0 to 1)
# 把修改後的像素壓在合理的範圍內, 因為像素不接受負的數值
perturbed_inputs = tf.clip_by_value(perturbed_inputs, 0, 1)
print(perturbed_inputs)

hack_img = perturbed_inputs.numpy()
hack_img = hack_img*255
hack_img = hack_img.reshape(28,28)

In [ ]:
hack_img = hack_img.astype(np.uint8)
im = Image.fromarray(hack_img)
im.save("hack.bmp")

test_data = np.asarray(hack_img)
plt.imshow(test_data,cmap='gray')